In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

!pip install pandas==1.0.3
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import scispacy
import en_core_sci_lg

import nltk
from nltk.corpus import stopwords

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
nlp = en_core_sci_lg.load()

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
treat_synonyms = [
    'treat',
    'treatment', 
    'alleviate', 
    'manage', 
    'suppress',
    'suppression', 
    'prescribe',
    'therapy',
    'cure',
    'remedy', 
    'candidate', 
    'therapeutic',
    'administer',
    'deliver',
    'inject',
    'take',
    'effect',
    'inhale',
    'inhalation',
    'administration',
    'rescue',
    'reduce',
    'improve']

In [ ]:
treat_vectors = {treat_synonym: nlp(treat_synonym).vector for treat_synonym in treat_synonyms}

In [ ]:
directory = '../input/coronawhy/v6_text/v6_text/'
filenames = os.listdir(directory)

In [ ]:
words_vectors = {}

for filename in filenames:
    filepath = os.path.join(directory, filename)
    print(filepath)
    df = pd.read_pickle(filepath, 'gzip')
    df['sentence'] = df['sentence'].str.lower()
    df['sentence'] = df['sentence'].str.replace('[{}]'.format(string.punctuation), '')
    df['sentence'] = df['sentence'].str.replace('|'.join(stop_words), '')
    
    for i, sentence in enumerate(df['sentence']):
        if not isinstance(sentence, str):
            continue
        for word in sentence.split():
            if word not in nlp.vocab:
                continue
            if word in words_vectors:
                continue

            vector = nlp(word).vector
            words_vectors[word] = vector

In [ ]:
words_vectors = pd.DataFrame(words_vectors).T
words_vectors = words_vectors[~(words_vectors == 0).all(axis=1)]

In [ ]:
words_vectors.to_csv('treawords_vectors.csv')

# Cosine Similarity

Perhaps the easiest way of measuring the similarity between vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df_similar_words = {}
for treat_word, vector in treat_vectors.items():
    word_cosine_similarity = cosine_similarity(X=words_vectors, Y=np.atleast_2d(vector))
    similar_words = pd.DataFrame(word_cosine_similarity, index=words_vectors.index).sort_values(by=0, ascending=False).head(10)[0]
    df_similar_words[treat_word] = similar_words

In [ ]:
df_similar_words

After going through the top-10 most similar words per synonym, I created a list of unique words per synonym thatI thought would be helpful.

In [ ]:
unique_similar_words = {
    'treat': ['curb', 'expel', 'help', 'prune'],
    'treatment': [],
    'alleviate': [],
    'manage': ['expunge'],
    'suppress': ['engulf'],
    'suppression': [],
    'prescript': ['refer'],
    'therapy': [],
    'cure': ['fluence'],
    'remedy': [],
    'candidate': [],
    'therapeutic': ['new', 'newer'],  # treatments talked about in research papers may be described as "new" or "newer",
    'administer': [],
    'deliver': [],
    'inject': [],
    'take': [],
    'effect': [],
    'inhale': ['huff', 'puff', 'gulp'],
    'inhalation': ['burner'],
    'administration': [],
    'rescue': [],
    'reduce': ['fewer', 'quench'],
    'improve': []
}

# Other Ideas

* Different language models:
    * BERT / ELMo that can create word vectors that change based on the context of the sentence.
    * using co-occurence/mutual information. e.g. if the word "treat" occurs, what other words appear alongside it?
* Finding better measurements of similarity:
    * KNN
    * Agglomerative clustering: similar to KNN but applied to clusters
    * t-SNE then clustering
    * PCA then clustering